# Background

In the past, my only experience with Spark was through Databricks in the 'Introduction to Spark' [EdX class](https://www.edx.org/course/introduction-apache-spark-uc-berkeleyx-cs105x). From my experience there, I found that the databricks environment was similar to that of Jupyter notebooks, and the syntax and operations of pyspark <strong>dataframes</strong>  were similar to what I'd use in SQL and Pandas. Thus, I decided that it was a good time to test it out locally. In this walkthrough, I hope to be able to load some Big Data (22 gigs) and see if I can perform some standard operations, and perhaps even some simple machine learning using MLLib.

## Loading Data

I moved my 22 gig csv file (courtesy of [Kaggle](https://www.kaggle.com/c/acquire-valued-shoppers-challenge/data?transactions.csv.gz) into my spark folder. Recall that in order to read a csv file with spark, you need to call it with a flag from terminal (even for notebook). In this case, it would be the following:

bin/pyspark --packages com.databricks:spark-csv_2.10:1.5.0

In [12]:
#Let's doublecheck that SparkContext is loaded on our machine
sc

In addition to using the flag from terminal, I also have to specifiy the databricks package from the options function in order to read the csv file properly into a pyspark dataframe.

In [7]:
data= sqlContext.read.format('com.databricks.spark.csv').options(
    header='true', inferschema='true').load('kaggle_shoppers_transactions.csv')

In [9]:
data.show(20)

+-----+-----+----+--------+----------+-----+----------+-----------+--------------+----------------+--------------+
|   id|chain|dept|category|   company|brand|      date|productsize|productmeasure|purchasequantity|purchaseamount|
+-----+-----+----+--------+----------+-----+----------+-----------+--------------+----------------+--------------+
|86246|  205|   7|     707|1078778070|12564|2012-03-02|       12.0|            OZ|               1|          7.59|
|86246|  205|  63|    6319| 107654575|17876|2012-03-02|       64.0|            OZ|               1|          1.59|
|86246|  205|  97|    9753|1022027929|    0|2012-03-02|        1.0|            CT|               1|          5.99|
|86246|  205|  25|    2509| 107996777|31373|2012-03-02|       16.0|            OZ|               1|          1.99|
|86246|  205|  55|    5555| 107684070|32094|2012-03-02|       16.0|            OZ|               2|         10.38|
|86246|  205|  97|    9753|1021015020|    0|2012-03-02|        1.0|            C

Sweet! It took about 5 min to 'load', but I was able to get it in there :) Let's check that the object is a dataframe.

In [10]:
type(data)

pyspark.sql.dataframe.DataFrame

In [ ]:
Operations

In [8]:
dataDF= data.toDF()
dataDF.show(5)

IllegalArgumentException: u"requirement failed: The number of columns doesn't match.\nOld column names (8): , userbank_id, account_currBal_count, account_currBal_median, account_currBal_mean, account_currBal_std, account_currBal_min, account_currBal_max\nNew column names (1): [userbank_id, account_currBal_count, account_currBal_median, account_currBal_mean, account_currBal_std, account_currBal_min, account_currBal_max]"

In [18]:
data2= data.toPandas()
type(data2)

pandas.core.frame.DataFrame

In [21]:
data.head(5)

[Row(=0, userbank_id=1454, account_currBal_count=2, account_currBal_median=51852.705, account_currBal_mean=51852.705, account_currBal_std=27445.2961227, account_currBal_min=32445.95, account_currBal_max=71259.46),
 Row(=1, userbank_id=30426, account_currBal_count=4, account_currBal_median=1323.56, account_currBal_mean=1991.67333333, account_currBal_std=1631.140728, account_currBal_min=800.66, account_currBal_max=3850.8),
 Row(=2, userbank_id=1453, account_currBal_count=5, account_currBal_median=345.76, account_currBal_mean=3520.564, account_currBal_std=5024.86835929, account_currBal_min=162.12, account_currBal_max=11680.32),
 Row(=3, userbank_id=7849, account_currBal_count=4, account_currBal_median=3583.05, account_currBal_mean=4327.5825, account_currBal_std=4577.36770741, account_currBal_min=0.0, account_currBal_max=10144.23),
 Row(=4, userbank_id=39068, account_currBal_count=4, account_currBal_median=5.0, account_currBal_mean=186.93, account_currBal_std=363.86, account_currBal_min=5.

In [25]:
data2= data.head(5)
type(data2)

list